In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# bertscore = load("bertscore") 
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")

/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")
import wandb
wandb.init(project="transformer_metric_finetuned_theres")
def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_inputs)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sanagrawal10401 (aparna02). Use `wandb login --relogin` to force relogin


In [6]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*10)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*10)
    eval_arg = [inpu,cands,lab]
    if i%1000 ==0:
        print(len(inpu),len(cands),len(lab))
        metric = metrics_func(eval_arg)
        print(metric)
        wandb.log({"bleu":metric[0],"sari":metric[1]})
        


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


10 10 10
10 10 10
3


  0%|          | 1/10000 [00:03<11:06:08,  4.00s/it]

(73.3092390467968, 33.67727861128607)


 10%|█         | 1000/10000 [27:47<2:48:42,  1.12s/it]

...

20010 20010 20010
20010 20010 20010
3


 20%|██        | 2001/10000 [43:35<16:07:51,  7.26s/it]

(61.235365065691774, 42.442474100220444)


 20%|██        | 2031/10000 [44:05<2:52:58,  1.30s/it] 


KeyboardInterrupt: 

In [ ]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*1)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*1)
    eval_arg = [inpu,cands,lab]
    if i%1000 ==0:
        print(len(inpu),len(cands),len(lab))
        metric = metrics_func(eval_arg)
        print(metric)
        wandb.log({"bleu":metric[0],"sari":metric[1]})

print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 34%|███▍      | 3376/10000 [2:11:50<3:37:24,  1.97s/it] 

In [4]:
text = "Since,2010, project researchers have uncovered documents in portugal that have revealed who owned the ship."
    # encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
    no_repeat_ngram_size=3,
    num_return_sequences=10,
    top_k=4, top_p=0.95
    # return_dict_in_generate=True,
)
for x, beam in enumerate(beam_outputs):
    print("{} {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))

NameError: name 'max_l' is not defined

In [ ]:
text = "Since,2010, project researchers have uncovered documents in portugal that have revealed who owned the ship."
    # encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
    no_repeat_ngram_size=3,
    num_return_sequences=10,
    top_k=4, top_p=0.95
    # return_dict_in_generate=True,
)
for x, beam in enumerate(beam_outputs):
    print("{} {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))

NameError: name 'tokenizer' is not defined

In [ ]:
text = "Experts say China's air pollution exacts a tremendous toll on human health."
    # encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
    no_repeat_ngram_size=3,
    num_return_sequences=10,
    top_k=4, top_p=0.95
    # return_dict_in_generate=True,
)
for x, beam in enumerate(beam_outputs):
    print("{} {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))